In [1]:
]activate .

In [2]:
using DifferentialEquations
using StaticArrays
using BenchmarkTools

@inbounds @inline function ż(z, p, t)
    A, B, D = p
    p₀, p₂ = z[SVector{2}(1:2)]
    q₀, q₂ = z[SVector{2}(3:4)]

    return SVector{4}(
        -A * q₀ - 3 * B / √2 * (q₂^2 - q₀^2) - D * q₀ * (q₀^2 + q₂^2),
        -q₂ * (A + 3 * √2 * B * q₀ + D * (q₀^2 + q₂^2)),
        A * p₀,
        A * p₂
    )
end

@inbounds @inline function ṗ(p, q, params, t)
    A, B, D = params
    dp1 = -A * q[1] - 3 * B / √2 * (q[2]^2 - q[1]^2) - D * q[1] * (q[1]^2 + q[2]^2)
    dp2 = -q[2] * (A + 3 * √2 * B * q[1] + D * (q[1]^2 + q[2]^2))
    return SVector{2}(dp1, dp2)
end

@inbounds @inline function q̇(p, q, params, t)
    params.A * p
end

q̇ (generic function with 1 method)

In [3]:
q0 = SVector{2}([0.0, -4.363920590485035])
p0 = SVector{2}([10.923918825236079, -5.393598858645495])
z0 = vcat(p0, q0)
p = (A=1,B=0.55,D=0.4)

tspan = (0., 10.)

(0.0, 10.0)

In [4]:
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 10.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [5]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10);

  849.914 μs (39946 allocations: 1.55 MiB)
  369.888 μs (17753 allocations: 534.38 KiB)
  1.499 ms (110196 allocations: 3.05 MiB)


In [6]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  789.911 μs (38974 allocations: 1.21 MiB)
  355.283 μs (17439 allocations: 463.05 KiB)
  1.438 ms (108191 allocations: 2.76 MiB)


In [7]:
function integ_benchmark(prob; args...)
    integ = init(prob; args...)
    while integ.t < prob.tspan[2]
        step!(integ)
    end
end

integ_benchmark (generic function with 1 method)

In [9]:
@btime integ_benchmark($prob1, alg=Vern9(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=KahanLi8(), dt=1e-2, maxiters=1e10)

  897.680 μs (40428 allocations: 1.55 MiB)
  379.758 μs (17909 allocations: 536.55 KiB)
  1.563 ms (111196 allocations: 3.06 MiB)


In [11]:
tspan = (0., 100.)
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 100.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [12]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  7.940 ms (386014 allocations: 11.80 MiB)
  3.480 ms (173907 allocations: 4.43 MiB)
  17.513 ms (1080083 allocations: 27.47 MiB)


In [13]:
@btime integ_benchmark($prob1, alg=Vern9(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime integ_benchmark($prob2, alg=KahanLi8(), dt=1e-2, maxiters=1e10)

  8.980 ms (400491 allocations: 15.50 MiB)
  3.749 ms (178553 allocations: 4.83 MiB)
  18.589 ms (1110082 allocations: 29.61 MiB)
